# Лабораторная работа 4. Рекомендательные системы и категориальные признаки

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

### Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — **15 баллов**. Сдавать задание после указанного срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).

Обратите внимание, что мы не ставим оценку за просто написанный код, корректная работоспособность которого не подтверждена экспериментами. Работы, в которых удалены формулировки заданий, проверять сложнее, поэтому **за решения без формулировок заданий мы будем снижать баллы в два раза**.

### Правила сдачи

Выполненную работу следует отправить в систему Anytask. Более подробно о системе можно почитать на странице курса. Название отправляемого файла должно иметь следующий формат: Surname_Name_Group_NN.ipynb, где NN — номер лабораторной работы. Например, Kozlova_Anna_CS_04.ipynb.

## Рекомендательные системы

![](http://i.imgur.com/vHUVoWw.png)

В этой лабораторной работе будет рассмотрена задача предсказания оценки, которую пользователь поставит фильму. Особенность этой задачи в том, что объекты выборки описываются категориальными признаками, принимающими большое число значений (например: идентификатор пользователя, идентификатор фильма, тэги, киноперсоны).

Мы будем работать с датасетом [MovieLens + IMDb/Rotten Tomatoes](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip) ([описание](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-readme.txt)). Набор содержит данные о предпочтениях пользователей сервиса рекомендации кинофильмов [MovieLens](http://www.movielens.org/). Пользовательские оценки для фильмов принимают целые значения в интервале от 1 до 5, они записаны в файле *user_ratedmovies.dat* (а так же в *user_ratedmovies-timestamps.dat*,  где для каждой оценки записаны дата и время в формате timestamp), остальные файлы содержат дополнительную информацию о фильмах, которую можно использовать в качестве признаков. Заметьте: кроме оценок (и тегов), про пользователя ничего не известно.

На основании этих данных необходимо построить модель, предсказывающую оценку пользователя фильму, который он еще не смотрел.

## Оценивание качества рекомендаций

Выберем некоторого пользователя $u$ и обозначим известные для него рейтинги за $R^u$. В качестве тестовых рейтингов этого пользователя $R^u_{test}$ рассмотрим три рейтинга, поставленные последними по времени. Остальные известные рейтинги будут составлять обучающую выборку $R^u_{train}$. Тогда все известные рейтинги можно представить как $R^u=R^u_{train}\cup R^u_{test}$. Отсутствующие оценки обозначим за $R^u_{unknown}$. 

Для измерения качества рекомендаций в этой лабораторной работе используйте две метрики RMSE и MAP, описанные ниже.

#### RMSE

Метрика [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation) вычисляется следующим образом:
$$ RMSE = \sqrt{ \frac{1}{\left|R_{test}\right|} \sum_{(u,i) \in R_{test}} (r_{ui} - \hat{r}_{ui})^2 },$$
где $r_{ui}$ — наблюдаемая (правильная) оценка, а $\hat{r}_{ui}$ — оценка, предсказанная моделью.

Метрика RMSE предназначена для оценки точности предсказания, ее удобно оптимизировать напрямую. Однако, нужно учесть, что RMSE не лучший кандидат для оценки качества рекомендаций:
- Во-первых, RMSE оценивает точность предсказания рейтингов, что в задачах рекомендаций, как правило, менее важно, нежели верное ранжирование объектов (безотносительно абсолютных значений рейтингов). На практике часто бывает, что алгоритмы с худшим RMSE в продакшене работают лучше, чем алгоритмы с меньшим значением RMSE. Например, при использовании моделей рекомендаций на основе матричных разложений, заполнение отсутствующих оценок нулями имеет имеет худшее значение RMSE чем если бы эти оценки не учитывались совсем, но при этом часто сами рекомендации становятся лучше. 
- Во-вторых, RMSE одинаково штрафует точность предсказания оценок более популярных фильмов (которые попадут в блок рекомендаций) и менее популярных фильмов (длинный хвост из нерелевантных фильмов).

#### MAP

Для оценки качества рекомендаций также можно использовать метрику качества ранжирования. В этом случае для каждого пользователя $u$ предскажем оценку для всех фильмов из $R^u_{test}$ и $R^u_{unknown}$ и отсортируем эти фильмы по убыванию предсказанного рейтинга. Ожидается, что хороший алгоритм должен выдать релевантные фильмы вверху списка. Обозначим позиции объектов в этом списке за $k^u_i$.

Назовем релевантными те фильмы, которые входят в $R^u_{test}$ и имеют оценку $\ge 3$. Обозначим их за $Rel^u$. Тогда можно вычислить следующую метрику качества рекомендаций для одного пользователя:

$$AP^u=\frac{1}{|Rel^u|} \sum_{(u,i) \in Rel^u} \frac{1}{k^u_i}.$$

Усреднив значение этой метрики по всем пользователями, мы получим окончательное значение метрики $MAP$. Пользователей без релевантных фильмов в тестовой выборке можно не учитывать.

#### Другие способы оценки качества рекомендаций

На практике, как правило, качество рекомендательных систем оценивается в онлайне с помощью [A/B-тестирования](https://ru.wikipedia.org/wiki/A/B-тестирование).

## Коллаборативная фильтрация

Загрузите данные и создайте разреженную матрицу пользователи-фильмы, где в каждой ячейке стоит рейтинг, если он известен, или ноль, если неизвестен.

### Most popular

**Задание 1 (1 балл)** Постройте рекоммендации на основе **most popular** метода, при котором пользователям рекомендуются объекты в порядке убываниях их популярности (например, среднего рейтинга). Оцените качество рекомендаций с использованием метрики MAP.

Для дальнейшей работы нам понадобится библиотека [Graphlab Create](https://turi.com/products/create/).

Чтобы получить доступ до дистрибутива необходима лицензия. Бесплатная лицензия на 1 месяц будет выслана вам на почту после регистрации. Для получения более долгой бесплатной лицензии для использования в академических целях, необходимо заполнить [форму](https://turi.com/download/academic.html).

Чтобы установить библиотеку нужно выполнить следующие шаги:

 - Зарегистрироваться по [ссылке](https://turi.com/download/academic.html). После регистрации получаете лицензию и Product key (эта информация будет продублирована в письме на указанную при регистрации почту). 
 - Далее, сразу после регистрации вы попадаете на страницу, с которой можно скачать дистрибутив для вашей системы и установить его. Либо, после регистрации, можно установить библиотеку через pip командой:

Подробнее про установку можно так же почитать [здесь](https://turi.com/download/install-graphlab-create-command-line.html).

Теперь рассмотрим [memory-based](https://en.wikipedia.org/wiki/Collaborative_filtering) методы рекоммендаций. Подход, лежащий в их основе, использует данные о рейтингах для вычисления сходства между пользователями (user-based) или объектами (item-based). На основе этих данных делаются предсказания рейтингов и, в дальнейшем, строятся рекомендации. Эти методы просты в реализации и эффективны на ранних стадиях разработки рекомендательных систем.

### Memory-based

**Задание 2 (1 балл)** Постройте рекоммендации на основе [item-based](https://turi.com/products/create/docs/generated/graphlab.recommender.item_similarity_recommender.ItemSimilarityRecommender.html) подхода, реализованном в библиотеке Graphlab Create. Оцените качество рекомендаций в зависимости от выбранной функции похожести *cosine/pearson* по каждой из описанных выше метрик (RMSE, MAP).

### Matrix factorization

**Задание 3 (2 балла)** Разложите матрицу рейтингов с помощью [разреженного SVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) и, восстановив ее, получите предсказания рейтингов для всех пар пользователь-объект. В данном случае неизвестные рейтинги заполняются нулями, а затем восстанавливаются с помощью SVD (этот метод называется PureSVD). Рассмотрите 5 различных значений ранга разложения. Оцените качество рекомендаций, используя описанные выше метрики. Для обеих метрик постройте графики зависимости качества алгоритма от выбранного ранга разложения. 

Как значение ранга влияет на метрики? Не забывайте делать аргументированные выводы на основании полученных результатов.

**Задание 4 (3 балла)** В пакете GraphLab Create также есть встроенный алгоритм рекомендации  [RankingFactorizationRecommender](https://turi.com/products/create/docs/generated/graphlab.recommender.ranking_factorization_recommender.RankingFactorizationRecommender.html) ([о том как его создавать](https://turi.com/products/create/docs/generated/graphlab.recommender.ranking_factorization_recommender.create.html#graphlab.recommender.ranking_factorization_recommender.create)). Проведите эксперименты из предыдущего задания для этого алгоритма рекомендаций, рассмотрев как влияют различные параметры на результат. Сравните их с полученными ранее результатами.

Описание параметров алгоритма можно найти [здесь](https://turi.com/products/create/docs/generated/graphlab.recommender.ranking_factorization_recommender.create.html). Мы рекомендуем при проведении экспериментов обратить внимание на некоторые из следующих параметров:
 - binary_targets
 - num_factors
 - ranking_regularization
 - unobserved_rating_value
 - num_sampled_negative_examples

## Категориальные и разреженные признаки

В этой части задания мы рассмотрим несколько подходов к работе с категориальными и разреженными признаками. В данном случае это id-пользователя и id-фильма, а также предлагаем на ваше усмотрение добавить один или несколько признаков из имеющихся данных, например: жанр фильма, киноперсоны из фильма, последний оцененный пользователем фильм, средняя оценка пользователя, ...

Для выбранного набора/наборов признаков будет необходимо построить рекомендательные системы, используя библиотеки [Vopwal Wabbit](https://github.com/JohnLangford/vowpal_wabbit/wiki), [LibFM](http://libfm.org) и [xgboost](https://github.com/dmlc/xgboost). При работе с этими библиотеками не забывайте, что консольные комманды нужно исполнять прямо из ipython-notebook'а, чтобы история ваших запусков сохранилась и проверяющие могли её оценить.

### Разреженные признаки

В данной части раьоты вам необходимо создать разреженную матрицу данных, закодировав каждый из категориальных признаков вектором чисел. Примером может служить следующая иллюстрация добавления различных признаков:

![](http://i.imgur.com/7nUMFx5.png)

Здесь, для наглядности, вся матрица объект-признак разбита на части, каждая из которых соответствует одному или группе категориальных признаков. Например, часть *User* соответствует закодированному признаку *user_id*, часть *Movie* — признаку *movie_id*, *Other movies rated* содержит в себе оценки пользователя другим фильмам, а *Last movie rated* соответствует признаку "последний оцененный пользователем фильм".

#### Vowpal Wabbit

С этой библиотекой вы знакомились в рамках другой лабораторной работы, где были даны подробне инструкции по ее установке и функциональности.

Создайте описанную выше выборку в формате пригодном для обучения модели с помощью Vopwal Wabbit.

** Задание 7 (2 балла)** Обучите **квадратичную модель** на полученной выбороке c помощью Vopwal Wabbit. Посчитайте качество предсказаний.

### libFm

Библиотека [libFM](http://libfm.org) реализует общий подход к задачам с категориальными признаками, в основе которого лежат факторизационные машины. В данном дистрибутиве реализованы такие методы как stochastic gradient descent ([SGD](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)), alternating least squares ([ALS](https://arxiv.org/pdf/1206.2493.pdf)), а также Bayesian inference using Markov Chain Monte Carlo ([MCMC](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo)). 
Подробную информацию можно найти в [мануале](http://www.libfm.org/libfm-1.42.manual.pdf).

libFm принимает данные в следующем формате:

    <target> <feature-id>:<feature-value> <feature-id>:<feature-value> ...

Например:

    4 0:1.5 3:-7.9
    2 1:1e-5 3:2
    -1 6:1

Если признак не указан, то он считается равным нулю. Таким образом, закодированный категориальный признак можно задать одной записью, например: 

    <user-id>:1.0

Создайте описанную выше выборку в формате пригодном для LibFM.

**Задание 8 (2 балла)** Постройте предсказания оценок при помощи факторизационных машин, используя библиотеку LibFM. Выберете наиболее подходящий метод обучения. Посчитайте качество построенной модели.

### Признаки-счётчики

В качестве иллюстрации что такое признаки-счётчики, рассмотрим категориальный признак "жанр фильма". Для каждого жанра мы можем посчитать некоторое числовое значение, например, среднее значение оценок фильмов этого жанра. Затем, если в матрице данных заменить значения жанров этого категориального признака на соответствующие значения средних рейтингов данных жанров, то получится новый числовой признак-счётчик.

Здесь стоит обратить внимание на следующее:

1. При создании таких счётчиков категориальных признаков по целевой переменной, важно не использовать оценки из настоящего и будущего. То есть, в нашем примере, при расчете среднего для конкретного объекта нельзя использовать как оценку текущего объекта, так и оценки, которые были поставлены позже. Иначе возникнет переобучение.
2. В качестве счетчиков можно рассматривать и другие статистики: число встречаемости данного значения, медиану целевой переменной по объектам с тем же значением данного категориального признака и т.п.
3. Подобные признаки-счётчики можно считать не только по одному категориальному признаку, как в примере с жанром фильма, но и и по набору из нескольких категориальных признаков, например, по паре (жанр, киноперсоны).
4. Счётчики можно считать не только по целевой переменной, но и относительно других признаков.

**Задание 8 (3 балла)** Используя исходные данные, создайте выборку с набором признаков-счётчиков. На полученной выборке с счетчиками постройте предсказания оценок, используя xgboost. Постарайтесь добиться качества сравнимого с качеством моделей из предыдущего пункта. Какие признаки-счётчики оказались наиболее удачными? Почему? 

**Задание 9 (1 балл)** Приведите сравнение качества всех моделей, использованных в работе, руководствуясь значениями описанных метрик. Какие из моделей оказались лучше других по каждой из метрик? Чем это можно объяснить?